In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Set random seed for reproducibility
torch.manual_seed(42)

# Load the Iris dataset and normalize the data
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Define the feedforward neural network class
class SimpleFeedforwardNN(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(SimpleFeedforwardNN, self).__init__()
        self.hidden1 = nn.Linear(input_dim, hidden_dim1)
        self.hidden2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.output = nn.Linear(hidden_dim2, output_dim)

    def forward(self, x):
        x = torch.relu(self.hidden1(x))
        x = torch.relu(self.hidden2(x))
        x = torch.softmax(self.output(x), dim=1)  # Use softmax for multi-class classification
        return x

# Initialize the model
input_dim = 28 * 28  # MNIST image size
hidden_dim1 = 128
hidden_dim2 = 64
output_dim = 10  # Three classes: Cat, Dog, Bird
model = SimpleFeedforwardNN(input_dim, hidden_dim1, hidden_dim2, output_dim)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Training loop
epochs = 10
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs = inputs.view(inputs.shape[0], -1)  # Flatten the images
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}")

print("Finished Training")

Epoch 1, Loss: 1.8069225792437473
Epoch 2, Loss: 1.5674625004786673
Epoch 3, Loss: 1.5506653964900767
Epoch 4, Loss: 1.540332635709726
Epoch 5, Loss: 1.5301734006989485
Epoch 6, Loss: 1.523090796938329
Epoch 7, Loss: 1.5173357699725674
Epoch 8, Loss: 1.5115640836992243
Epoch 9, Loss: 1.507377313398349
Epoch 10, Loss: 1.5041535201865728
Finished Training


In [51]:
import cv2
from tkinter import Tk, filedialog

# Global variables
line_points = []
image = None
clone = None

def select_image():
    root = Tk()
    root.withdraw()  # Hide the main window

    file_path = filedialog.askopenfilename()  # Open a file dialog to choose an image
    return file_path

def draw_lines(event, x, y, flags, param):
    global line_points, image

    if event == cv2.EVENT_LBUTTONDOWN:
        line_points.append((x, y))

        if len(line_points) == 2:
            cv2.line(image, line_points[0], line_points[1], (0, 0, 255), 8)
            line_points = []  # Reset for the next line

            cv2.imshow("Draw Lines", image)

def main():
    global image, clone

    image_path = select_image()

    if image_path:
        image = cv2.imread(image_path)
        clone = image.copy()

        cv2.namedWindow("Draw Lines", cv2.WINDOW_NORMAL)
        cv2.setMouseCallback("Draw Lines", draw_lines)

        while True:
            cv2.imshow("Draw Lines", image)
            key = cv2.waitKey(1) & 0xFF

            if key == ord("d"):
                image = cv2.resize(image, (0, 0), fx=0.5, fy=0.5)

            elif key == ord("i"):
                image = cv2.resize(image, (0, 0), fx=2, fy=2)

            # Press 's' to save the modified image
            elif key == ord("s"):
                save_path = filedialog.asksaveasfilename(defaultextension=".jpg", filetypes=[("JPEG files", "*.jpg")])
                if save_path:
                    cv2.imwrite(save_path, image)
                    print(f"Image saved as {save_path}")

            # Press 'r' to reload the original image
            elif key == ord("r"):
                image = clone.copy()

            elif key == 27:
                break

        cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Image saved as /home/ali/Downloads/SS.jpg


In [3]:
import cv2
import numpy as np
import math
from tkinter import Tk, filedialog

def select_image():
    root = Tk()
    root.withdraw()  # Hide the main window

    file_path = filedialog.askopenfilename()  # Open a file dialog to choose an image
    return file_path

def calculate_angle(line1, line2):
    angle_radians = math.atan2(line2[1] - line1[1], line2[0] - line1[0])
    angle_degrees = math.degrees(angle_radians)
    return angle_degrees

image_path = select_image()

if image_path:
    image = cv2.imread(image_path)
    image_copy = image.copy()

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_red = np.array([0, 50, 50])
    upper_red = np.array([10, 255, 255])
    mask = cv2.inRange(hsv, lower_red, upper_red)

    edges = cv2.Canny(mask, threshold1=50, threshold2=150)
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=50, maxLineGap=5)

    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(image, (x1, y1), (x2, y2), (0, 0, 255), 2)

    cv2.imshow("Detected Red Lines", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    angles = []
    for i in range(len(lines)):
        for j in range(i + 1, len(lines)):
            angle = calculate_angle((lines[i][0][0], lines[i][0][1]), (lines[j][0][0], lines[j][0][1]))
            angles.append(angle)

    print("Angles between detected red lines:")
    for idx, angle in enumerate(angles):
        print(f"Pair {idx + 1}: {angle} degrees")


Angles between detected red lines:
Pair 1: 88.74554837731847 degrees
Pair 2: -86.63353933657021 degrees
Pair 3: 92.64254529406472 degrees
Pair 4: -89.12532322887868 degrees
Pair 5: 91.84167349474727 degrees
Pair 6: -89.28084388345493 degrees
Pair 7: 93.8310513610567 degrees
Pair 8: -89.8564018557688 degrees
Pair 9: 94.27278076760925 degrees
Pair 10: 92.75008517491241 degrees
Pair 11: -90.71615994547041 degrees
Pair 12: 92.21858173383949 degrees
Pair 13: -87.90392359280195 degrees
Pair 14: -86.44970324171197 degrees
Pair 15: 91.39961841954033 degrees


In [56]:
import cv2
import numpy as np
import math
import tkinter as tk
from tkinter import filedialog

def select_image():
    root = tk.Tk()
    root.withdraw()  # Hide the main window

    file_path = filedialog.askopenfilename()  # Open a file dialog to choose an image
    return file_path

def calculate_angle(line1, line2):
    angle_radians = math.atan2(line2[1] - line1[1], line2[0] - line1[0])
    angle_degrees = math.degrees(angle_radians)
    return angle_degrees

image_path = select_image()

if image_path:
    image = cv2.imread(image_path)
    image_copy = image.copy()

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_red = np.array([0, 50, 50])
    upper_red = np.array([10, 255, 255])
    mask = cv2.inRange(hsv, lower_red, upper_red)

    edges = cv2.Canny(mask, threshold1=50, threshold2=150)
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=20, minLineLength=150, maxLineGap=15)
    #lines = cv2.HoughLinesPointSet(edges, rho_res=1, lines_max=4, theta_res=np.pi / 180, threshold=100)
    #lines = cv2.HoughLinesPointSet(edges, lines_max=20, threshold=5, min_rho=0, max_rho=20, rho_step=1,min_theta=0, max_theta=np.pi, theta_step=np.pi/100)


    for idx, line in enumerate(lines):
        x1, y1, x2, y2 = line[0]
        cv2.line(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
        edge_number_position = ((x1 + x2) // 2, (y1 + y2) // 2)
        cv2.putText(image, str(idx), edge_number_position, cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    angles = []
    angle_info = []

    for i in range(len(lines)):
        for j in range(i + 1, len(lines)):
            angle = calculate_angle((lines[i][0][0], lines[i][0][1]), (lines[j][0][0], lines[j][0][1]))
            angles.append(angle)
            angle_info.append((i, j))  # Store edge indices for angle pairs

    for angle, (edge_idx1, edge_idx2) in zip(angles, angle_info):
        x1, y1 = lines[edge_idx1][0][0], lines[edge_idx1][0][1]
        x2, y2 = lines[edge_idx2][0][0], lines[edge_idx2][0][1]
        text_position = ((x1 + x2) // 2, (y1 + y2) // 2)
        cv2.putText(image, f"{angle:.2f}°", text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        start_point = (lines[edge_idx1][0][0], lines[edge_idx1][0][1])
        end_point = (lines[edge_idx2][0][0], lines[edge_idx2][0][1])
        #cv2.putText(image, f"Start: {start_point}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
        #cv2.putText(image, f"End: {end_point}", (x2, y2 + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

    window_name = "Detected Red Lines"
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)  # Create a resizable window
    cv2.imshow(window_name, image)

    while True:
        key = cv2.waitKey(1) & 0xFF

        # Press 'esc' to exit
        if key == 27:
            break

    cv2.destroyAllWindows()


In [61]:
import cv2
import numpy as np
import math
import tkinter as tk
from tkinter import filedialog

def select_image():
    root = tk.Tk()
    root.withdraw()  # Hide the main window

    file_path = filedialog.askopenfilename()  # Open a file dialog to choose an image
    return file_path

def calculate_slope_degree(point1, point2):
    angle_radians = math.atan2(point2[1] - point1[1], point2[0] - point1[0])
    angle_degrees = math.degrees(angle_radians)
    return angle_degrees

image_path = select_image()

if image_path:
    image = cv2.imread(image_path)
    image_copy = image.copy()

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_red = np.array([0, 50, 50])
    upper_red = np.array([10, 255, 255])
    mask = cv2.inRange(hsv, lower_red, upper_red)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        if cv2.contourArea(contour) > 50:  # Filter out small contours
            epsilon = 0.1 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)

            if len(approx) == 2:  # Line segments have 2 points
                start_point = tuple(approx[0][0])
                end_point = tuple(approx[1][0])

                slope_degree = calculate_slope_degree(start_point, end_point)
                midpoint = ((start_point[0] + end_point[0]) // 2, (start_point[1] + end_point[1]) // 2)

                cv2.circle(image_copy, start_point, 5, (0, 255, 0), -1)  # Green circle at start point
                cv2.circle(image_copy, end_point, 5, (0, 255, 0), -1)    # Green circle at end point
                cv2.putText(image_copy, f"{slope_degree:.2f}°", midpoint, cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    window_name = "Slope Degrees and Points of Red Lines"
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)  # Create a resizable window
    cv2.imshow(window_name, image_copy)

    while True:
        key = cv2.waitKey(1) & 0xFF

        # Press 'esc' to exit
        if key == 27:
            break

    cv2.destroyAllWindows()
